In [190]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [146]:
dataset_ref = pd.read_csv('census_starter.csv')
counties_ref = gpd.read_file('us-county-boundaries.csv')
train = pd.read_csv('train.csv')


In [147]:
train["county"] = train['county'].str.replace(' County', '')
train["county"] = train['county'].str.replace(' Parish', '')
train["county"] = train['county'].str.replace(' Municipality', '')
train["county"] = train['county'].str.replace(' Borough', '')
train["county"] = train['county'].str.replace(' Census Area', '')
train["county"] = train['county'].str.replace(' City', '')
train["county"] = train['county'].str.replace(' city', '')
train["county"] = train['county'].str.replace(' and', '')

train["County State"] = train["county"] + " " + train["state"]

train

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,County State
0,1001_2019-08-01,1001,Autauga,Alabama,2019-08-01,3.007682,1249,Autauga Alabama
1,1001_2019-09-01,1001,Autauga,Alabama,2019-09-01,2.884870,1198,Autauga Alabama
2,1001_2019-10-01,1001,Autauga,Alabama,2019-10-01,3.055843,1269,Autauga Alabama
3,1001_2019-11-01,1001,Autauga,Alabama,2019-11-01,2.993233,1243,Autauga Alabama
4,1001_2019-12-01,1001,Autauga,Alabama,2019-12-01,2.993233,1243,Autauga Alabama
...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston,Wyoming,2022-06-01,1.803249,101,Weston Wyoming
122261,56045_2022-07-01,56045,Weston,Wyoming,2022-07-01,1.803249,101,Weston Wyoming
122262,56045_2022-08-01,56045,Weston,Wyoming,2022-08-01,1.785395,100,Weston Wyoming
122263,56045_2022-09-01,56045,Weston,Wyoming,2022-09-01,1.785395,100,Weston Wyoming


In [148]:
counties_ref["longitude"] = counties_ref["Geo Point"].apply(lambda x: x.split(",")[0]).astype(float)
counties_ref["latitude"] = counties_ref["Geo Point"].apply(lambda x: x.split(",")[1]).astype(float)
counties_ref.head()

,Geo Point,Geo Shape,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,STUSAB,LSAD,...,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,STATE_NAME,COUNTYFP NOZERO,geometry,longitude,latitude
0,"41.001938375, -83.6665354077","{""coordinates"": [[[-83.880762, 41.080361], [-8...",39,063,01074044,39063,Hancock,Hancock County,OH,06,...,A,1376125660,6020647,+41.0004711,-083.6660335,Ohio,63,None,41.001938,-83.666535
1,"38.4207190191, -77.4574003866","{""coordinates"": [[[-77.588783, 38.504044], [-7...",51,179,01480173,51179,Stafford,Stafford County,VA,06,...,A,697237202,28088011,+38.4132608,-077.4513342,Virginia,179,None,38.420719,-77.457400
2,"40.1764332141, -98.4999473753","{""coordinates"": [[[-98.726624, 40.29186], [-98...",31,181,00835912,31181,Webster,Webster County,NE,06,...,A,1489017962,337241,+40.1806461,-098.4985897,Nebraska,181,None,40.176433,-98.499947
3,"28.4225359982, -99.756726125","{""coordinates"": [[[-99.780854, 28.643827], [-9...",48,127,01383849,48127,Dimmit,Dimmit County,TX,06,...,A,3441795367,14576535,+28.4235871,-099.7658713,Texas,127,None,28.422536,-99.756726
4,"41.7723266607, -91.1324230468","{""coordinates"": [[[-91.366079, 41.860068], [-9...",19,031,00465205,19031,Cedar,Cedar County,IA,06,...,A,1500798712,6385832,+41.7723600,-091.1326105,Iowa,31,None,41.772327,-91.132423


In [149]:
counties_ref["County State"] = counties_ref["NAME"] + " " + counties_ref["STATE_NAME"]
counties_ref.set_index("County State", inplace=True)
train["County State"].loc[train["County State"].str.contains("Ana New Mexico")] = "Dona Ana New Mexico"


/tmp/ipykernel_43227/836429503.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["County State"].loc[train["County State"].str.contains("Ana New Mexico")] = "Dona Ana New Mexico"


In [150]:
def find_lon(county_state):
    try:
        return counties_ref.loc[county_state]["longitude"]
    except:
        return None
    
def find_lat(county_state):
    try:
        return counties_ref.loc[county_state]["latitude"]
    except:
        return None
    
train["longitude"] = train["County State"].apply(find_lon)
train["latitude"] = train["County State"].apply(find_lat)

In [151]:
from geopy.geocoders import Nominatim
import shapely

geolocator = Nominatim(user_agent="My app")

def fetch_coordinates(name: str, country: str) -> tuple:
    geo = geolocator.geocode(f"{name}, {country}")
    try:
        return (geo.point.latitude, geo.point.longitude)
    except:
        return (None, None)

missing_counties = train.loc[train["longitude"].isna()]["County State"].unique()

for county in missing_counties:
    coordinates = fetch_coordinates(county, "USA")
    train.loc[train["County State"] == county, "latitude"] = coordinates[0]
    train.loc[train["County State"] == county, "longitude"] = coordinates[1]

In [152]:
train.loc[train["longitude"].isna()]["County State"].unique()

array([], dtype=object)

In [167]:
train["first_day_of_month"] = pd.to_datetime(train["first_day_of_month"])

In [188]:
maps_files = os.listdir("data")
maps_files.remove("us_map.npy")

maps = {map.replace(".npy", ""):np.load(f"data/{map}") for map in maps_files}